<a href="https://colab.research.google.com/github/AjeetSingh02/Notebooks/blob/master/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%%capture
!pip install confuse

In [0]:
from describe import describe

In [0]:
import pandas as pd
import numpy as np

df = pd.read_csv("/content/train.csv")
for col in df.columns:
    if df[col].dtype == 'object':
        try:
            df[col] = pd.to_datetime(df[col])
        except ValueError:
            pass

In [0]:
df.shape

(2935849, 6)

In [0]:
import time

In [0]:
#pandas profiling

start_time = time.time()
profile = pandas_profiling.ProfileReport(df)
parent_dict = profile.get_description()
end_time = time.time()
print("--- %s pandas profiling seconds ---" % (end_time - start_time))

--- 71.49362516403198 pandas profiling seconds ---


In [0]:
df = pd.read_csv("/content/sample_data/mnist_train_small.csv")

In [0]:
df.shape

(19999, 785)

In [0]:
#without modin

start_time = time.time()
parent_dict = describe(df)
end_time = time.time()

print("--- %s without modin seconds ---" % (end_time - start_time))

--- 48.31317591667175 without modin seconds ---


In [0]:
%%capture
!pip install pandas_profiling==2.1.0

In [0]:
%%capture
!pip install simplejson==3.16.0

In [0]:
import pandas_profiling

In [0]:
# !pip show pandas_profiling

In [0]:
!python caller_v16.py '/content/sample_data/california_housing_test.csv' '/content/out1' '/content/possible_keys1.yml'

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.switch_backend('agg')


#for numerical columns
def plotbox(series, output_folder, count, whis):
    fig1, ax1 = plt.subplots(figsize=(14,7))
    ax1.grid(b=None)
    plt.grid(b=None)

    ax1.set(xlabel=series.name)
    
    ret_dict = ax1.boxplot(series, vert=False, widths=0.8, patch_artist=True, sym ="b", whis=whis)

    median = ret_dict["medians"][0]._x[0]
    q1 = ret_dict["boxes"][0]._path._vertices[0][0]
    q3 = ret_dict["boxes"][0]._path._vertices[2][0]
    whisker_cap1 = ret_dict["caps"][0]._x[0]
    whisker_cap2 = ret_dict["caps"][1]._x[0]

    loc_q1 = ret_dict["boxes"][0]._path._vertices[1][1]
    loc_q2 = ret_dict["boxes"][0]._path._vertices[0][1]
    loc_whisk1 = ret_dict["caps"][0]._y[1]
    loc_whisk2 = ret_dict["caps"][1]._y[0]

    clr = 'red'
    fsize = 12
    ax1.text(median, 1.03, f"median = {median:,.2f}", fontsize=fsize, color=clr)
    ax1.text(q1, loc_q1+0.02, f"1st Quarter = {q1:,.2f}",  fontsize=fsize, color=clr)
    ax1.text(q3, loc_q2-0.02, f"3rd Quarter = {q3:,.2f}",  fontsize=fsize, color=clr)
    ax1.text(whisker_cap1, loc_whisk1+0.02, f"leftmost inlier = {whisker_cap1:,.2f}",  fontsize=fsize, color=clr)
    ax1.text(whisker_cap2, loc_whisk2-0.02, f"rightmost inlier = {whisker_cap2:,.2f}",  fontsize=fsize, color=clr)

    try:
        plot_name = series.name
    except:
        count += 1
        plot_name = f"boxplot_{count}"


    #calculation for no. and percent of outliers
    low_outlier = np.where(series < whisker_cap1)[0]
    high_outlier = np.where(series > whisker_cap2)[0]
    num_outliers = low_outlier.size + high_outlier.size
    perc_outliers = num_outliers / len(series)


    ax1.set_title(f"Number of outliers: {num_outliers}          Percent of Outliers: {round(perc_outliers,2)}")

    fig1.savefig(f"{output_folder}{plot_name}.png")
    plt.close()
    
    return count


#For "string" type variables
def plotCountCat(series, output_folder, count, unique_class_limit):
  try:
    class_count = len(series.value_counts())
  except AttributeError as e:
    print(f"Series object not passed. Following exception encountered: {str(e)}")
    return count
  
  if class_count <= unique_class_limit:
    x = class_count+4 if class_count < 11 else 15 
    
    fig, ax = plt.subplots(figsize=(x, 6))
    plt.xticks(rotation=90)
    
    ax1 = sns.countplot(x=series, palette="Set3")

    
    try:
      plot_name = series.name
    except:
      count += 1
      plot_name = f"catplot_{count}"

    ax1.set(xlabel=plot_name, ylabel='Number of occurances')

    fig.savefig(f"{output_folder}{plot_name}.png", bbox_inches = "tight")

    plt.close()

  return count


def plots(df, output_folder, whis, unique_class_limit):
    count = 0
    for i in range(len(df.columns)):
        series = df.iloc[:,i]
        dropped_na = series.dropna()

        if not dropped_na.empty:
            if series.dtype == "O":
                count = plotCountCat(dropped_na, output_folder, count, unique_class_limit)
            
            elif dropped_na.dtype == "datetime64[ns]":
                fst = dropped_na.iloc[0]
                lst = dropped_na.iloc[0]
                mid = dropped_na.iloc[dropped_na.size//2]

                lis_ = [fst.hour, fst.minute, fst.second, lst.hour, lst.minute, lst.second, mid.hour, mid.minute, mid.second]

                #checking if datetime64 has empty time. if yes remove it
                if all([ele == 0 for ele in lis_]):
                    count = plotCountCat(dropped_na.dt.date, output_folder, count, unique_class_limit)
                else:
                    count = plotCountCat(dropped_na, output_folder, count, unique_class_limit)
                
            else:
                if sorted(list(dropped_na.unique())) != [0,1]:
                    count = plotbox(dropped_na, output_folder, count, whis)

In [0]:
import yaml
import simplejson
import numpy as np
import pandas as pd
import pandas_profiling

import matplotlib.pyplot as plt
plt.switch_backend('agg')


#function to act as converter when default json converter fails
def convert(o):
    if isinstance(o, np.int64): 
        return int(o) 
    elif isinstance(o, np.bool_):
        if o == True:
            return True
        return False
    elif isinstance(o, pd.Timestamp):
        if o.hour == 0 and o.minute == 0 and o.second == 0:
            return o.strftime("%d-%m-%Y")
        else:
            return str(o)

def save_json(df, output_folder, yaml_file, file_name):

    #saving original names and underscored names
    original_cols= list(df.columns)
    replacer = lambda t: t.replace(" ","_")
    vfunc = np.vectorize(replacer)
    replaced_cols = list(vfunc(original_cols))
    reference_cols = {k:v for k,v in zip(replaced_cols, original_cols)}


    profile = pandas_profiling.ProfileReport(df)
    parent_dict = profile.get_description()

    data = {}
    data["overview"] = parent_dict["table"]
    variables = parent_dict["variables"]

    with open(yaml_file, 'r') as stream:
        try:
            possible_keys = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            return f"following error occured while loading yaml file: {exc}"

    data["variables general info"] = {}
    data["variables general info"]["header"] = possible_keys

    data["variables frequency info"] = {}
    data["variables frequency info"]["value_counts_without_nan"] = {}

    #For info of each variable
    for variable, dict_ in variables.items():
        temp_list = []
        for key in possible_keys:
            if key in dict_:
                #for dealing with enum
                if key == "type":
                    temp_list.append(dict_[key].value)
                else:  
                    temp_list.append(dict_[key])
            else:
                temp_list.append("")
        
        #getting original variable name
        try:
            variable = reference_cols[variable]
        except KeyError:
            pass

        data["variables general info"][variable] = temp_list

        #for frequency
        without_nan = dict_["value_counts_without_nan"]

        index, values = list(without_nan.index), list(without_nan.values)

        if len(index) > 10:
            index = index[:5] + index[-5:]
            values = values[:5] + values[-5:]
        
        data["variables frequency info"]["value_counts_without_nan"][variable] = [index, values]


    try:
        correlations = parent_dict["correlations"]

        # data["correlations"] = {}
        correlations_dict = {}

        for corr, df_ in correlations.items():
            df_ = df_.round(2)
            df_.fillna("", inplace=True)

            #getting original variable names
            pp_cols = df_.columns
            orig_cols= []
            for col in pp_cols:
                try:
                    orig_cols.append(reference_cols[col])
                except KeyError:
                    orig_cols.append(col)

            correlations_dict[corr] = [orig_cols, [list(val) for val in df_.values]]

        with open(f"{output_folder}{file_name}_corr_.json", "w") as write_file:
            simplejson.dump(correlations_dict, write_file, default=convert, ignore_nan=True, separators=(',', ':')) 
    except:
        print("No correlation generated")
        
    with open(f"{output_folder}{file_name}.json", "w") as write_file:
        simplejson.dump(data, write_file, default=convert, ignore_nan=True, separators=(',', ':')) 	

In [0]:
import pandas as pd
import pandas_profiling

df = pd.read_csv("/content/saa.csv")
output_folder = "/content/out/"
whis = None
unique_class_limit = 35
yaml_file = "/content/possible_keys1.yml"
file_name = "nothing"

for col in df.columns:
    if df[col].dtype == 'object':
        try:
            df[col] = pd.to_datetime(df[col])
        except ValueError:
            pass

# plots(df, output_folder, whis, unique_class_limit)
data = save_json(df, output_folder, yaml_file, file_name)

In [0]:
# import yaml
# import simplejson
# import numpy as np
# import pandas as pd
# import pandas_profiling

# import seaborn as sns
# import matplotlib.pyplot as plt
# plt.switch_backend('agg')

# def caller(input_file, output_folder, yaml_file, whis=None, unique_class_limit=35):
#   if not output_folder.endswith("/"):
#     output_folder += "/"

#   df = pd.read_csv(input_file)

#   file_name = input_file[input_file.rindex("/")+1:input_file.rindex(".")]

#   #Extracting information from DataFrame using pandas-profiling
#   save_json(df, output_folder, yaml_file, file_name)

#   #making boxplots of all non-categorical and non-binary columns of DataFrame
#   plots(df, output_folder, whis, unique_class_limit)


In [0]:
'''Support for ISO-8859-1 added'''


import pandas as pd
from sys import argv

from plots_v15 import plots
from auto_eda_v29 import save_json

if __name__ == '__main__':
    input_file, output_folder, yaml_file  = argv[1:4]
    outlier_limit = None
    unique_class_limit = 35

    if len(argv) > 4:
        try:
            outlier_limit = float(argv[4])
        except ValueError:
            print("Note: Wrong value for outlier limit. Set to default: None")
        
        try:
            unique_class_limit = float(argv[5])
        except IndexError:
            pass
        except ValueError:
            print("Note: Wrong value for unique_class_limit. Set to default: 35")
        
    if not output_folder.endswith("/"):
        output_folder += "/"
    
    try:
        df = pd.read_csv(input_file)
    except UnicodeDecodeError:
        df = pd.read_csv(input_file, encoding="ISO-8859-1")
        
    
    for col in df.columns:
        if df[col].dtype == 'object':
            try:
                df[col] = pd.to_datetime(df[col])
            except ValueError:
                pass

    
    file_name = input_file[input_file.rindex("/")+1:input_file.rindex(".")]

    option = "1"
    if option == "1":
        #to plot boxplots for numerical/continuous variables. and bar plot categorical
        plots(df, output_folder, outlier_limit, unique_class_limit)

        #use pandas profiling extract details and save in json
        save_json(df, output_folder, yaml_file, file_name)	
        

In [0]:
input_file = "/content/train.csv"
output_folder = "/content/out6"
yaml_file = '/content/possible_keys.yml'

caller(input_file, output_folder, yaml_file)

In [0]:
df = pd.read_csv("/content/saa.csv")

In [0]:
for col in df.columns:
    if df[col].dtype == 'object':
        try:
            df[col] = pd.to_datetime(df[col])
        except ValueError:
            pass

In [0]:
for col in df.columns:
    print(df[col].name)
    print(df[col].dtype == "datetime64[ns]")
    print()

date
False

timestamp
True

total
False

tt
True

